In [3]:
import numpy as np
import random
import os
import glob
import pandas as pd

In [4]:
def drop_outlier(array,count,bins):
    index = []
    range_ = np.arange(1,count,bins)
    for i in range_[:-1]:
        array_lim = array[i:i+bins]
        sigma = np.std(array_lim)
        mean = np.mean(array_lim)
        th_max,th_min = mean + sigma*2, mean - sigma*2
        idx = np.where((array_lim < th_max) & (array_lim > th_min))
        idx = idx[0] + i
        index.extend(list(idx))
    return np.array(index)

In [ ]:
Battery_list = ['CS2_35', 'CS2_36', 'CS2_37', 'CS2_38']

dir_path = 'datasets/CALCE/'
Battery = {}
for name in Battery_list:
    print('Load Dataset ' + name + ' ...')
    path = glob.glob(dir_path + name + '/*.xlsx')
    dates = []
    for p in path:
        df = pd.read_excel(p, sheet_name=1)
        print('Load ' + str(p) + ' ...')
        dates.append(df['Date_Time'][0])
    idx = np.argsort(dates)
    path_sorted = np.array(path)[idx]
    
    count = 0
    discharge_capacities = []
    health_indicator = []
    internal_resistance = []
    CCCT = []
    CVCT = []
    for p in path_sorted:
        df = pd.read_excel(p,sheet_name=1)
        print('Load ' + str(p) + ' ...')
        cycles = list(set(df['Cycle_Index']))
        for c in cycles:
            df_lim = df[df['Cycle_Index'] == c]
            #Charging
            df_c = df_lim[(df_lim['Step_Index'] == 2)|(df_lim['Step_Index'] == 4)]
            c_v = df_c['Voltage(V)']
            c_c = df_c['Current(A)']
            c_t = df_c['Test_Time(s)']
            #CC or CV
            df_cc = df_lim[df_lim['Step_Index'] == 2]
            df_cv = df_lim[df_lim['Step_Index'] == 4]
            CCCT.append(np.max(df_cc['Test_Time(s)'])-np.min(df_cc['Test_Time(s)']))
            CVCT.append(np.max(df_cv['Test_Time(s)'])-np.min(df_cv['Test_Time(s)']))

            #Discharging
            df_d = df_lim[df_lim['Step_Index'] == 7]
            d_v = df_d['Voltage(V)']
            d_c = df_d['Current(A)']
            d_t = df_d['Test_Time(s)']
            d_im = df_d['Internal_Resistance(Ohm)']

            if(len(list(d_c)) != 0):
                time_diff = np.diff(list(d_t))
                d_c = np.array(list(d_c))[1:]
                discharge_capacity = time_diff*d_c/3600 # Q = A*h
                discharge_capacity = [np.sum(discharge_capacity[:n]) for n in range(discharge_capacity.shape[0])]
                discharge_capacities.append(-1*discharge_capacity[-1])

                dec = np.abs(np.array(d_v) - 3.8)[1:]
                start = np.array(discharge_capacity)[np.argmin(dec)]
                dec = np.abs(np.array(d_v) - 3.4)[1:]
                end = np.array(discharge_capacity)[np.argmin(dec)]
                health_indicator.append(-1 * (end - start))

                internal_resistance.append(np.mean(np.array(d_im)))
                count += 1

    discharge_capacities = np.array(discharge_capacities)
    health_indicator = np.array(health_indicator)
    internal_resistance = np.array(internal_resistance)
    CCCT = np.array(CCCT)
    CVCT = np.array(CVCT)
    
    idx = drop_outlier(discharge_capacities, count, 40)
    df_result = pd.DataFrame({'cycle':np.linspace(1,idx.shape[0],idx.shape[0]),
                              'capacity':discharge_capacities[idx],
                              'SoH':health_indicator[idx],
                              'resistance':internal_resistance[idx],
                              'CCCT':CCCT[idx],
                              'CVCT':CVCT[idx]})
    Battery[name] = df_result

Load Dataset CS2_35 ...
Load datasets/CALCE/CS2_35/CS2_35_10_29_10.xlsx ...
Load datasets/CALCE/CS2_35/CS2_35_1_10_11.xlsx ...
Load datasets/CALCE/CS2_35/CS2_35_2_4_11.xlsx ...
Load datasets/CALCE/CS2_35/CS2_35_1_28_11.xlsx ...
Load datasets/CALCE/CS2_35/CS2_35_10_15_10.xlsx ...
Load datasets/CALCE/CS2_35/CS2_35_12_20_10.xlsx ...
Load datasets/CALCE/CS2_35/CS2_35_11_24_10.xlsx ...
Load datasets/CALCE/CS2_35/CS2_35_8_18_10.xlsx ...
Load datasets/CALCE/CS2_35/CS2_35_1_18_11.xlsx ...
Load datasets/CALCE/CS2_35/CS2_35_12_06_10.xlsx ...
Load datasets/CALCE/CS2_35/CS2_35_1_24_11.xlsx ...
Load datasets/CALCE/CS2_35/CS2_35_12_23_10.xlsx ...
Load datasets/CALCE/CS2_35/CS2_35_8_17_10.xlsx ...
Load datasets/CALCE/CS2_35/CS2_35_11_08_10.xlsx ...
Load datasets/CALCE/CS2_35/CS2_35_9_8_10.xlsx ...
Load datasets/CALCE/CS2_35/CS2_35_2_10_11.xlsx ...
Load datasets/CALCE/CS2_35/CS2_35_8_19_10.xlsx ...
Load datasets/CALCE/CS2_35/CS2_35_10_22_10.xlsx ...
Load datasets/CALCE/CS2_35/CS2_35_9_7_10.xlsx ...
Lo

In [9]:
Battery['CS2_35']

,cycle,capacity,SoH,resistance,CCCT,CVCT
0,1.0,1.126385,0.825175,0.094009,6613.059052,2251.498033
1,2.0,1.126160,0.815965,0.091661,6612.402800,2231.967052
2,3.0,1.125966,0.815977,0.094649,6608.560673,2228.216959
3,4.0,1.118508,0.825194,0.091413,6604.732222,2247.561061
4,5.0,1.117210,0.806900,0.091413,6629.211049,2077.692393
...,...,...,...,...,...,...
877,878.0,0.339218,0.128353,0.121454,1255.711238,NaN
878,879.0,0.330025,0.128343,0.122852,1535.380556,3328.912803
879,880.0,0.330027,0.119175,0.120863,1291.478395,3022.668035
880,881.0,0.320860,0.119175,0.122264,1290.025300,3007.262188
